Reinforcement Learning for Large Language Models
Winter 23/24 Semester
Final Group Project
Kateryna Smykovska, Jakob Schmitter, Suvi Lehtosalo, Megan Horikawa

This notebook was made by **Megan Horikawa** for 7b, expanded and ran on 13b by Kateryna Smykovska (marked by "by KS")

In [1]:
!pip install transformers
!pip install accelerate
!pip install bitsandbytes
!pip install sentencepiece
!pip install torch
!pip install datasets
!pip install evaluate
!pip install huggingface_hub

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.1/290.1 kB 4.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 23.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 36.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 15.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 14.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.0/196.0 MB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 166.0/166.0 MB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.1/99.1 kB 15.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━

In [3]:
from transformers import LlamaForCausalLM, LlamaTokenizer, AutoTokenizer, BitsAndBytesConfig
import torch

import datasets
import evaluate
import numpy as np

from google.colab import userdata
my_secret_key = userdata.get('HF_TOKEN')

# config for
BnB_config= BitsAndBytesConfig(
                load_in_4bit=True,
                bnb_4bit_compute_dtype=torch.bfloat16,
                bnb_4bit_use_double_quant=True,
                bnb_4bit_quant_type='nf4'
                )

tokenizer = AutoTokenizer.from_pretrained("meta-llama/Llama-2-13b-hf", token=my_secret_key, quantization_config=BnB_config)

model = LlamaForCausalLM.from_pretrained(
    "meta-llama/Llama-2-13b-hf",
    device_map="auto",
    token = my_secret_key
)

device=torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

tokenizer_config.json:   0%|          | 0.00/776 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/610 [00:00<?, ?B/s]

...

model-00001-of-00003.safetensors:   0%|          | 0.00/9.95G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/9.90G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/6.18G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/188 [00:00<?, ?B/s]

In [4]:
# Load Dataset for QA task. Superglue subdataset copa was chosen
copa_dataset = datasets.load_dataset("super_glue", "copa")

#train subset was selected as it had over 150 entries with a mixture of both labels
dataset = copa_dataset['train'].shuffle(seed=42).select(range(30))


Generating train split:   0%|          | 0/400 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/100 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/500 [00:00<?, ? examples/s]

In [ ]:
# define log likelihood function (taken from homework with small changes for llama)

def get_log_prob_of_completion(
        model,
        tokenizer,
        prompt,
        completion,
        device=torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
):
        """
        Convenience function for computing the log probability of a completion
        given a prompt.
        """
        # tokenize the prompt and the completion
        # truncate so as to fit into to maximal context window of llama2
        # which is 2048 tokens
        input_ids = tokenizer(
                prompt + completion,
                return_tensors='pt',
                truncation=True,
                max_length=2048,
        )['input_ids'].to(device)

        # separately tokenize prompt
        # so as to access the logits for the completion only
        # when scoring the completion
        input_ids_prompt = tokenizer(
                prompt,
                return_tensors='pt',
                truncation=True,
                max_length=2048
        )['input_ids'].to(device)

        # create attention mask and position ids
        attention_mask = (input_ids != tokenizer.eos_token_id).to(dtype=torch.int64)
        position_ids = attention_mask.cumsum(-1)-1
        # get the logits for the completion
        with torch.no_grad():
                out = model(
                        input_ids=input_ids,
                        attention_mask=attention_mask,
                        position_ids=position_ids
                )

        # get the logits of the completion
        # for that, make a tensor of the logits
        # for the completion only
        # in particular, we shift the indices by one to the left to access logits of the
        # actual sequence tokens
        logits_completion = out.logits[:, :-1]
        logits_completion = logits_completion.squeeze()
        # get the log probabilities for the completion
        log_probs = torch.nn.functional.log_softmax(
                logits_completion,
                dim=-1
        )
        # retrieve the logit corresponding to the actual completion tokens
        try:
                log_completion_tokens = log_probs.gather(
                        dim=-1,
                        index=input_ids[:, 1:].squeeze().unsqueeze(-1)
                )
        except:
                log_completion_tokens = log_probs.gather(
                        dim=-1,
                        index=input_ids[:, 1:].unsqueeze(-1)
                )

        continuationConditionalLogProbs = log_completion_tokens[
                (input_ids_prompt.shape[-1]-1):
        ]
        completion_log_prob = torch.mean(
                continuationConditionalLogProbs
        ).cpu()

        return completion_log_prob

In [ ]:
#iterate over the dataset:

results = []
prompt_list = []
correct_option = []
incorrect_option = []
log_ps_correct_option = []
log_ps_incorrect_option = []


def make_question(question):
    if question == 'cause':
        return "What is the cause of this?"
    if question == 'effect':
        return "What is the effect of this?"
idx = 1

for item in dataset:
  # concatenate the premise with each of the choices
    prompt = item['premise'] + make_question(item['question'])
  # get classification of item as either entailment (0) or non-entailment(1)
    category = item['label']

    correct = ''
    incorrect = ''

  # set correct classification based on label
    if category == 0:
        correct = item['choice1']
        incorrect = item['choice2']

    else:
        correct = item['choice2']
        incorrect = item['choice1']

    prompt_list.append(prompt)
    correct_option.append(correct)
    incorrect_option.append(incorrect)

  #compute the lob probabilities for both choices
    log_p_correct = get_log_prob_of_completion(
      model,
      tokenizer,
      prompt,
      correct
    )
  # append to the correct list
    log_ps_correct_option.append(log_p_correct)

    log_p_incorrect = get_log_prob_of_completion(
      model,
      tokenizer,
      prompt,
      incorrect
  )

  # append to the incorrect list
    log_ps_incorrect_option.append(log_p_incorrect)
    print(f'finished {idx} of {len(dataset)}')
    idx += 1

In [ ]:
# lets load things into pandas
import pandas as pd

df = pd.DataFrame(list(zip(prompt_list,correct_option, log_ps_correct_option,incorrect_option, log_ps_incorrect_option)), columns = ['prompt','correct', 'log_prob_correct','incorrect', 'log_prob_incorrect'])


# evaluate the log probabilities
# check whether the log probaility of the correct answer is higher than the
# incorrect answer and append to the results list
df['correct_prediction'] = df['log_prob_correct']> df['log_prob_incorrect']


df.head(20)


In [ ]:
# export to csv

df.to_csv('QA_llama2_13b.csv', index=False)

Link to csv file: https://drive.google.com/file/d/1hjInJMNmndr3XBWZGXyx4XHHVxz_5Uj_/view?usp=drive_link

In [ ]:
df.groupby('correct_prediction').count()

In [ ]:
#disconnect from runtime
# from google.colab import runtime
# runtime.unassign()

In [ ]:
#by KS
import csv
def generate_zero_shot_responses(dataset, temperature=0.7):
  responses = []
  for item in dataset:
    prompt = f"""You are given the premise and two sentences with it. Determine which sentence is a cause/effect for the premise.
    If the answer is the first sentence, answer with 0. If the answer is the second sentence, answer with 1.\n
    Premise: {item["premise"]}.\n
    To determine: {item["question"]}.\n
    Sentences: {item["choice1"] + " " + item["choice2"]}\n
    Answer:
    """
    input_ids = tokenizer(prompt, return_tensors="pt").input_ids.to(model.device)
    output = model.generate(input_ids, max_new_tokens=1, temperature=temperature)
    generated_text = tokenizer.decode(output[0], skip_special_tokens=True)
    responses.append(generated_text)

  return responses

results_zero_shot = generate_zero_shot_responses(dataset)

csv_file_path = "zero_shot_responses.csv"

data_zero_shot = []
for index, response in enumerate(results_zero_shot, start=1):
    premise = dataset[index - 1]["premise"]
    question = dataset[index - 1]["question"]
    choice1 = dataset[index - 1]["choice1"]
    choice2 = dataset[index - 1]["choice2"]
    label = dataset[index - 1]["label"]
    data_zero_shot.append((premise, question, choice1, choice2, label, response))

with open(csv_file_path, mode='w', newline='', encoding='utf-8') as file:
    writer = csv.writer(file)
    writer.writerow(['premise', 'question', 'choice1', 'choice2', 'label', 'generated_response'])
    writer.writerows(data_zero_shot)

In [5]:
#by KS
import csv
def generate_few_shot_2_examples(dataset):
  responses = []
  for item in dataset:
    prompt = f"""You are given the premise and two sentences with it. Determine which sentence is a cause/effect for the premise.
    If the answer is the first sentence, answer with 0. If the answer is the second sentence, answer with 1.\n
    Premise: The ocean tide was dangerous.\n
    To determine: effect \n
    Sentences: The swimmers put on more sunscreen. The swimmers returned towards the shore.\n
    Answer: 1\n
    Premise: I put shoes on.\n
    To determine: cause
    Sentences: The floor contained broken glass. The paint on the walls was wet.\n
    Answer: 0\n
    Premise: {item["premise"]}.\n
    To determine: {item["question"]}.\n
    Sentences: {item["choice1"] + " " + item["choice2"]}\n
    Answer:
    """
    input_ids = tokenizer(prompt, return_tensors="pt").input_ids.to(model.device)
    output = model.generate(input_ids, max_new_tokens=1)
    generated_text = tokenizer.decode(output[0], skip_special_tokens=True)
    responses.append(generated_text)

  return responses

results_few_shot_2_examples = generate_few_shot_2_examples(dataset)

csv_file_path = "few_shot_2_examples.csv"

data_few_shot_2_examples = []
for index, response in enumerate(results_few_shot_2_examples, start=1):
    premise = dataset[index - 1]["premise"]
    question = dataset[index - 1]["question"]
    choice1 = dataset[index - 1]["choice1"]
    choice2 = dataset[index - 1]["choice2"]
    label = dataset[index - 1]["label"]
    data_few_shot_2_examples.append((premise, question, choice1, choice2, label, response))

with open(csv_file_path, mode='w', newline='', encoding='utf-8') as file:
    writer = csv.writer(file)
    writer.writerow(['premise', 'question', 'choice1', 'choice2', 'label', 'generated_response'])
    writer.writerows(data_few_shot_2_examples)

In [ ]:
#by KS
def generate_few_shot_4_examples(dataset):
  responses = []
  for item in dataset:
    prompt = f"""You are given the premise and two sentences with it. Determine which sentence is a cause/effect for the premise.
    If the answer is the first sentence, answer with 0. If the answer is the second sentence, answer with 1.\n
    Premise: The ocean tide was dangerous.\n
    To determine: effect \n
    Sentences: The swimmers put on more sunscreen. The swimmers returned towards the shore.\n
    Answer: 1\n
    Premise: I put shoes on.\n
    To determine: cause
    Sentences: The floor contained broken glass. The paint on the walls was wet.\n
    Answer: 0\n
    Premise: The flashlight was dead.\n
    To determine: effect\n
    Sentences: I took it apart. I replaced the batteries.\n
    Answer: 1\n
    Premise: The boy felt homesick.\n
    To determine: cause\n
    Sentences: He went away to camp. He bickered with his sister.\n
    Answer: 0\n
    Premise: {item["premise"]}.\n
    To determine: {item["question"]}.\n
    Sentences: {item["choice1"] + " " + item["choice2"]}\n
    Answer:
    """
    input_ids = tokenizer(prompt, return_tensors="pt").input_ids.to(model.device)
    output = model.generate(input_ids, max_new_tokens=1)
    generated_text = tokenizer.decode(output[0], skip_special_tokens=True)
    responses.append(generated_text)

  return responses

results_few_shot_4_examples = generate_few_shot_4_examples(dataset)

csv_file_path = "few_shot_4_examples.csv"

data_few_shot_4_examples = []
for index, response in enumerate(results_few_shot_4_examples, start=1):
    premise = dataset[index - 1]["premise"]
    question = dataset[index - 1]["question"]
    choice1 = dataset[index - 1]["choice1"]
    choice2 = dataset[index - 1]["choice2"]
    label = dataset[index - 1]["label"]
    data_few_shot_4_examples.append((premise, question, choice1, choice2, label, response))

with open(csv_file_path, mode='w', newline='', encoding='utf-8') as file:
    writer = csv.writer(file)
    writer.writerow(['premise', 'question', 'choice1', 'choice2', 'label', 'generated_response'])
    writer.writerows(data_few_shot_4_examples)